In [1]:
sc.install_pypi_package('graphframes')
sc.install_pypi_package('pandas')
sc.install_pypi_package('numpy')
sc.install_pypi_package('seaborn')
sc.install_pypi_package('s3fs')
sc.setCheckpointDir("/tmp")

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
9,application_1592076919424_0010,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/0b/27/c7c7e1ced2fe9a905f865dd91faaec2ac8a8e313f511678c8ec92a41a153/graphframes-0.6-py2.py3-none-any.whl

  Using cached https://files.pythonhosted.org/packages/a4/5f/1b6e0efab4bfb738478919d40b0e3e1a06e3d9996da45eb62a77e9a090d9/pandas-1.0.4-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/d4/70/d60450c3dd48ef87586924207ae8907090de0b306af2bce5d134d78615cb/python_dateutil-2.8.1-py2.py3-none-any.whl


  Using cached https://files.pythonhosted.org/packages/c7/e6/54aaaafd0b87f51dfba92ba73da94151aa3bc179e5fe88fc5dfb3038e860/seaborn-0.10.1-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/dd/82/c1fe128f3526b128cfd185580ba40d01371c5d299fcf7f77968e22dfcc2e/scipy-1.4.1-cp37-cp37m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/b2/c2/71fcf957710f3ba1f09088b35776a799ba7dd95f7c2b195ec800933b276b/matplotlib-3.2.1-cp37-cp37m-manylinux1_x86_64.whl
  Using cach

In [2]:
from graphframes import *
import re
import s3fs
import math
import boto
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
import seaborn as sns
from pyspark.sql import SparkSession, Row
import pyspark.sql.functions as F
from pyspark.sql.functions import max as sqlmax
from pyspark.sql.functions import min as sqlmin
from pyspark.sql.functions import sum as sqlsum
from pyspark.sql.functions import mean as sqlmean
from graphframes.lib import AggregateMessages as AM

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

First all the required files are read from s3 buckets

In [3]:
# Data collection and cleaning already done, data uploaded to s3 bucket
# code has been uploaded as a separate python file;
df = pd.read_csv('s3://aws-emr-resources-825655629964-us-east-1/data_final3.csv')
tot_awards = []
for s in df['awards']:
    if isinstance(s,str):
        val=[int(u) for u in re.findall('[0-9]+',s)]
        val = sum(val)
    else:
        val = 0
    tot_awards.append(val)
df['tot_awards'] = tot_awards

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# Consolidating and extrapolating financial data
fin_new = pd.read_csv('s3://aws-emr-resources-825655629964-us-east-1/finance_final.csv')
fin_new.columns = ['name', 'world_gross', 'prod_bud']
fin_new.index = fin_new.name
df.index = df.title
ids = list(set(df.index).intersection(fin_new.index))
df.loc[ids, 'world_gross'] = fin_new.loc[ids, 'world_gross']
df.loc[ids, 'prod_bud'] = fin_new.loc[ids, 'prod_bud']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
ids1 = list(df[pd.notnull(df.world_gross) & pd.notnull(df.prod_bud)].index)
avg_g2p = df.loc[ids1, 'world_gross'].sum() / df.loc[ids1, 'prod_bud'].sum()

ids2 = list(df[(df.world_gross.isnull()) & pd.notnull(df.prod_bud)].index)
df.loc[ids2, 'world_gross'] = avg_g2p * df.loc[ids2, 'prod_bud']

ids3 = list(df[(df.prod_bud.isnull()) & pd.notnull(df.world_gross)].index)
df.loc[ids3, 'prod_bud'] = df.loc[ids3, 'world_gross'] / avg_g2p 

ids4 = list(set(df.index).difference(set(ids1+ids2+ids3)))
df.loc[ids4, 'prod_bud'] = df.prod_bud.mean()
df.loc[ids4, 'world_gross'] = df.world_gross.mean()

missing_wg = ids2+ids4
missing_pb = ids3+ids4

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Now I repurpose the data to create a graph

In [6]:
# Create Node List
vertices = pd.DataFrame()
vertices.loc[:, 'nodes'] = df['title']
vertices.loc[:, 'year'] = df['year']
vertices.loc[:, 'imdb_rating'] = df['imdb_rating']
vertices.loc[:, 'tot_awards'] = df['tot_awards']
vertices.loc[:, 'bechdel'] = df['bechdel']
vertices.loc[:, 'prod_bud'] = df['prod_bud']
vertices.loc[:, 'world_gross'] = df['world_gross']

vertices.loc[:, 'type'] = (['movie'] * vertices.shape[0])
vertices.loc[:, 'count'] = ([1] * vertices.shape[0])
movies = list(df['title'])
cat = ['director', 'actors', 'writer']
edges = []

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
for c in cat:
    titles = []
    regex = re.compile(".*?\((.*?)\)")
    new_names = list(df[c])
    new_names = [i.split(',') for i in new_names]
    titles = [[movies[ind]] * len(new_names[ind]) for ind in range(len(new_names))]
    
    new_names = reduce(lambda x, y: x + y, new_names)
    titles = reduce(lambda x, y: x + y, titles)
    new_names = [re.sub('\(.*?\)','', i) for i in new_names]
    new_names = [re.sub('"','', i) for i in new_names]
    new_names = [re.sub('\'','', i) for i in new_names]

    new_names = [i.strip() for i in new_names]
    edges += [i for i in zip(new_names, titles, [c]*len(new_names))]
    new_names = list(set(new_names))

    new_nodes = pd.DataFrame()
    new_nodes.loc[:, 'nodes'] = pd.Series(new_names)
    new_nodes.loc[:, 'type'] = pd.Series(['artist'] * len(new_names))
    new_nodes.loc[:, 'year'] = pd.Series([None] * len(new_names))
    new_nodes.loc[:, 'bechdel'] = pd.Series([None] * len(new_names))
    new_nodes.loc[:, 'prod_bud'] = pd.Series([None] * len(new_names))
    new_nodes.loc[:, 'world_gross'] = pd.Series([None] * len(new_names))
    
    new_nodes.loc[:, 'imdb_rating'] = pd.Series([None] * len(new_names))
    new_nodes.loc[:, 'tot_awards'] = pd.Series([None] * len(new_names))
    new_nodes.loc[:, 'count'] = pd.Series([1] * len(new_names))
    
    vertices = pd.concat([vertices, new_nodes])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
spark = SparkSession.builder.appName('pandasToSparkDF').getOrCreate()
vertices.drop_duplicates(subset=['nodes'], inplace=True)
vertices = spark.createDataFrame(vertices)
vertices = vertices.withColumnRenamed('nodes', 'id')

edges = [list(i) for i in edges]
edges = pd.DataFrame(edges)
edges.columns = ['person', 'movie', 'type']
edges = spark.createDataFrame(edges)
edges2 = (edges.withColumnRenamed('movie', 'src')
              .withColumnRenamed('person', 'dst')
        )
edges = (edges.withColumnRenamed('movie', 'dst')
              .withColumnRenamed('person', 'src')
        )

g = GraphFrame(vertices, edges)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Finally I move on to compute all the required features

In [9]:
# Construct final dataframe to feed into the ML pipeline
artists = g.filterVertices('type != "movie"').vertices.toPandas()['id']
mldata = pd.DataFrame(artists)
mldata.columns = ['artist']
mldata.index = list(mldata['artist'])
mldata.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                artist
Sam Fell                      Sam Fell
Larry Clark                Larry Clark
Taylor Hackford        Taylor Hackford
Michael A. Nickles  Michael A. Nickles
Scott Thomas              Scott Thomas

Output variables

In [10]:
# Work volume
df1 = g.filterVertices('type != "movie"').vertices
df1 = df1.dropDuplicates(subset=['id'])
df2 = g.outDegrees
df2 = df2.dropDuplicates(subset=['id'])
num_movies = df2.join(df1, df1.id == df2.id, 'inner').drop(df1.id).drop(df1.type).dropDuplicates(subset=['id']).toPandas()
mldata.loc[num_movies['id'],'num_movies'] = list(num_movies['outDegree'])
mldata.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                                artist  num_movies
Sam Fell                      Sam Fell           4
Larry Clark                Larry Clark           3
Taylor Hackford        Taylor Hackford           5
Michael A. Nickles  Michael A. Nickles           2
Scott Thomas              Scott Thomas           2

In [11]:
# Years active
msgToSrc = AM.dst["year"]
aggmax = g.aggregateMessages(
    sqlmax(AM.msg).alias("maxyear"),
    sendToSrc=msgToSrc).toPandas()
aggmin = g.aggregateMessages(
    sqlmin(AM.msg).alias("minyear"),
    sendToSrc=msgToSrc).toPandas()
aggmax.index = aggmax['id']
aggmin.index = aggmin['id']
mldata.loc[:,'yactive'] = aggmax.loc[mldata.index, 'maxyear']\
                        - aggmin.loc[mldata.index, 'minyear']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Quality features

In [12]:
# Rating - min and max IMDb ratings
msgToSrc = AM.dst["imdb_rating"]
aggmax = g.aggregateMessages(
    sqlmax(AM.msg).alias("maxrating"),
    sendToSrc=msgToSrc).toPandas()
aggmean = g.aggregateMessages(
    sqlmean(AM.msg).alias("meanrating"),
    sendToSrc=msgToSrc).toPandas()
aggmax.index = aggmax['id']
aggmean.index = aggmean['id']
mldata.loc[:,'maxrating'] = aggmax.loc[mldata.index, 'maxrating']
mldata.loc[:,'meanrating'] = aggmean.loc[mldata.index, 'meanrating']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
# Awards & nominations - total and mean
msgToSrc = AM.dst["tot_awards"]
aggmax = g.aggregateMessages(
    sqlmax(AM.msg).alias("maxawards"),
    sendToSrc=msgToSrc).toPandas()
aggmean = g.aggregateMessages(
    sqlmean(AM.msg).alias("meanawards"),
    sendToSrc=msgToSrc).toPandas()
aggmax.index = aggmax['id']
aggmean.index = aggmean['id']
mldata.loc[:,'maxawards'] = aggmax.loc[mldata.index, 'maxawards']
mldata.loc[:,'meanawards'] = aggmean.loc[mldata.index, 'meanawards']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# Bechdel - total and mean
msgToSrc = AM.dst["bechdel"]
aggmean = g.aggregateMessages(
    sqlmean(AM.msg).alias("meanbechdel"),
    sendToSrc=msgToSrc).toPandas()
aggmean.index = aggmean['id']
mldata.loc[:,'meanbechdel'] = aggmean.loc[mldata.index, 'meanbechdel']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
# Box office collection
msgToSrc = AM.dst["world_gross"]
aggmax = g.aggregateMessages(
    sqlmax(AM.msg).alias("maxbo"),
    sendToSrc=msgToSrc).toPandas()
aggmean = g.aggregateMessages(
    sqlmean(AM.msg).alias("meanbo"),
    sendToSrc=msgToSrc).toPandas()
aggmax.index = aggmax['id']
aggmean.index = aggmean['id']
mldata.loc[:,'maxbo'] = aggmax.loc[mldata.index, 'maxbo']
mldata.loc[:,'meanbo'] = aggmean.loc[mldata.index, 'meanbo']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Network features

In [16]:
# all connected co-artists
cc = g.connectedComponents()
coartists = {}
x = cc.toPandas()
x.index = x.id
for c in x.component.unique():
    search = "component={} and type='artist'".format(c)
    coartists[c] = cc.filter(search).count()
cinds = x.loc[mldata.index, 'component']
set([coartists[c] for c in cinds])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 18, 20, 12085}

I will not be using this feature because the graph essentially contains only one large connected component and hence won't give much additional information. Instead I will look at all artists within edge distance = 2 from an artist as her co-artists (i.e. someone she has worked with on any movie).

In [16]:
# Number of co-artists
msgToDst = AM.src["count"]
aggsum = g.aggregateMessages(
    sqlsum(AM.msg).alias("count1"),
    sendToDst=msgToDst).toPandas()
aggsum.index = aggsum['id']
vertices = vertices.toPandas()
vertices.index = vertices.id
vertices.loc[:, 'count1'] = pd.Series([0] * vertices.count())
vertices.loc[aggsum.index, 'count1'] = aggsum.count1
vertices = spark.createDataFrame(vertices)
gdash = GraphFrame(vertices, edges)
msgToSrc = AM.dst["count"]
aggsum = g.aggregateMessages(
    sqlsum(AM.msg).alias("count1"),
    sendToSrc=msgToSrc).toPandas()
aggsum.index = aggsum['id']
mldata.loc[:,'numcoartists'] = aggsum.loc[mldata.index, 'count1']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
# Co-artist influence
gdash = GraphFrame(vertices, edges2)
results = gdash.pageRank(resetProbability=0.15, tol=0.01)
pr = results.vertices.select("id", "pagerank")
pr = pr.toPandas()
pr.index = pr.id
mldata.loc[:,'pagerank'] = pr.loc[mldata.index, 'pagerank']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# Production company network
proddf = pd.read_csv('s3://aws-emr-resources-825655629964-us-east-1/production5.csv')
proddf.drop(columns=['budget'], inplace=True)
proddf2 = proddf.copy()
proddf2.columns = ['title', 'production']
edges_prod = spark.createDataFrame(pd.concat([proddf2, proddf]))
edges_prod = (edges_prod.withColumnRenamed('production', 'dst')
               .withColumnRenamed('title', 'src')
      )

vertices_prod = list(proddf.title) + list(proddf.production)
vertices_prod = pd.DataFrame(vertices_prod)
vertices_prod.columns = ['id']
vertices_prod.drop_duplicates(subset=['id'], inplace=True)
vertices_prod = spark.createDataFrame(vertices_prod)
g_prod = GraphFrame(vertices_prod, edges_prod)
results = g_prod.pageRank(resetProbability=0.15, tol=0.05)
pr = results.vertices.select("id", "pagerank")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
search = g.vertices.filter("type=='movie'").toPandas()
vertices = vertices.toPandas()
vertices.index = vertices.id
pr = pr.toPandas()
pr.index = pr.id
ids = list(set(search.id).intersection(set(pr.id)))
vertices.loc[:, 'pagerank'] = [0] * vertices.shape[0]
vertices.loc[ids, 'pagerank'] = pr.loc[ids, 'pagerank']
vertices = spark.createDataFrame(vertices)
gdash = GraphFrame(vertices, edges)
msgToSrc = AM.dst["pagerank"]
aggmax = gdash.aggregateMessages(
    sqlmax(AM.msg).alias("pagerank"),
    sendToSrc=msgToSrc).toPandas()
aggmax.index = aggmax['id']
mldata.loc[:,'pagerank'] = aggmax.loc[mldata.index, 'pagerank']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
# Production budget
msgToSrc = AM.dst["prod_bud"]
aggmax = g.aggregateMessages(
    sqlmax(AM.msg).alias("maxpb"),
    sendToSrc=msgToSrc).toPandas()
aggmean = g.aggregateMessages(
    sqlmean(AM.msg).alias("meanpb"),
    sendToSrc=msgToSrc).toPandas()
aggmax.index = aggmax['id']
aggmean.index = aggmean['id']
mldata.loc[:,'maxpb'] = aggmax.loc[mldata.index, 'maxpb']
mldata.loc[:,'meanpb'] = aggmean.loc[mldata.index, 'meanpb']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
mldata.to_csv('s3://aws-emr-resources-825655629964-us-east-1/mldatav1.csv', index=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Machine Learning

In [23]:
mldata = pd.read_csv('s3://aws-emr-resources-825655629964-us-east-1/mldatav1.csv')
print(mldata.columns)
print(mldata.head() )
mldata = spark.createDataFrame(mldata)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Index(['artist', 'num_movies', 'yactive', 'maxrating', 'meanrating',
       'maxawards', 'meanawards', 'meanbechdel', 'maxbo', 'meanbo',
       'numcoartists', 'pagerank', 'maxpb', 'meanpb'],
      dtype='object')
               artist  num_movies  ...         maxpb        meanpb
0            Sam Fell           4  ...  1.490000e+08  1.045000e+08
1         Larry Clark           3  ...  2.100000e+07  7.866667e+06
2     Taylor Hackford           5  ...  6.500000e+07  4.100000e+07
3  Michael A. Nickles           2  ...  7.500000e+06  7.500000e+06
4        Scott Thomas           2  ...  3.820879e+07  3.820879e+07

[5 rows x 14 columns]

In [24]:
from pyspark.ml.regression import LinearRegression, RandomForestRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
mldata.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['artist', 'num_movies', 'yactive', 'maxrating', 'meanrating', 'maxawards', 'meanawards', 'meanbechdel', 'maxbo', 'meanbo', 'numcoartists', 'pagerank', 'maxpb', 'meanpb']

In [26]:
def regression_model(features, mldata, label):
    mldata_train, mldata_test = mldata.randomSplit([0.8, 0.2], seed=17)

    assembler = VectorAssembler(inputCols=features, outputCol='features')
    regression = LinearRegression(labelCol=label)
    pipeline = Pipeline(stages=[assembler, regression])
    evaluator = RegressionEvaluator(labelCol=label)

    params = ParamGridBuilder()
    params = params.addGrid(regression.regParam, [0.01,0.1,1.0,10.0]) \
                   .addGrid(regression.elasticNetParam, [0.0,0.5,1.0])
    params = params.build()

    cv = CrossValidator(estimator=pipeline, estimatorParamMaps=params, evaluator=evaluator, numFolds=5)
    cv = cv.fit(mldata_train)
    best_model = cv.bestModel
    predictions = best_model.transform(mldata_test)
    rmse = evaluator.evaluate(predictions)
    
    return rmse

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
merit_feat = ['maxrating', 'meanrating', 'maxawards', 'meanawards', 'meanbechdel']
network_feat = ['maxbo', 'meanbo', 'numcoartists', 'pagerank', 'maxpb', 'meanpb']

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
print('model RMSE for predicting number of movies')
print('merit features:', regression_model(merit_feat, mldata, 'num_movies'))
print('network features:', regression_model(network_feat, mldata, 'num_movies'))
print('both merit and network features:', regression_model(merit_feat+network_feat, mldata, 'num_movies'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-28:
Traceback (most recent call last):
  File "/emr/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/emr/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 1698



model RMSE for predicting number of movies
merit features: 1.6869520786853196
network features: 0.009557625328660143
both merit and network features: 0.009557598165038265

In [29]:
print('model RMSE for predicting number of active years')
print('merit features:', regression_model(merit_feat, mldata, 'yactive'))
print('network features:', regression_model(network_feat, mldata, 'yactive'))
print('both merit and network features:', regression_model(merit_feat+network_feat, mldata, 'yactive'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-29:
Traceback (most recent call last):
  File "/emr/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/emr/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 2498



model RMSE for predicting number of active years
merit features: 2.4663624617820745
network features: 2.281160756643252
both merit and network features: 2.0501275918333253

In [30]:
def random_forest_model(features, mldata, label):
    mldata_train, mldata_test = mldata.randomSplit([0.8, 0.2], seed=17)

    assembler = VectorAssembler(inputCols=features, outputCol='features')
    forest = RandomForestRegressor(labelCol=label)
    pipeline = Pipeline(stages=[assembler, forest])
    evaluator = RegressionEvaluator(labelCol=label)

    params = ParamGridBuilder()
    params = params.addGrid(forest.featureSubsetStrategy, ['all', 'onethird', 'sqrt', 'log2']) \
                   .addGrid(forest.maxDepth, [2, 5, 10])
    params = params.build()

    cv = CrossValidator(estimator=pipeline, estimatorParamMaps=params, evaluator=evaluator, numFolds=5)
    cv = cv.fit(mldata_train)
    best_model = cv.bestModel
    predictions = best_model.transform(mldata_test)
    rmse = evaluator.evaluate(predictions)
    
    return rmse

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
print('model RMSE for predicting number of movies')
print('merit features:', random_forest_model(merit_feat, mldata, 'num_movies'))
print('network features:', random_forest_model(network_feat, mldata, 'num_movies'))
print('both merit and network features:', random_forest_model(merit_feat+network_feat, mldata, 'num_movies'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

model RMSE for predicting number of movies
merit features: 1.4779209749378392
network features: 0.0026028960314768046
both merit and network features: 0.013199091933711351

In [1]:
# Note that a network feature is strongly correlated to the outcome feature: i.e. number of co-artists with number of movies
# which is why the RMSE is so small, so need to re-run analysis by omitting this variable and maybe adding other measures

In [32]:
print('model RMSE for predicting number of active years')
print('merit features:', random_forest_model(merit_feat, mldata, 'yactive'))
print('network features:', random_forest_model(network_feat, mldata, 'yactive'))
print('both merit and network features:', random_forest_model(merit_feat+network_feat, mldata, 'yactive'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-31:
Traceback (most recent call last):
  File "/emr/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/emr/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 6910

Exception in thread cell_monitor-32:
Traceback (most recent call last):
  File "/emr/notebook-env/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/emr/notebook-env/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/emr/notebook-env/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell

model RMSE for predicting number of active years
merit features: 2.0539315406154155
network features: 1.8106673650277643
both merit and network features: 1.7969651322570954